# Prenotazione Hotel con Middleware per Membri Prioritari

Questo notebook dimostra il **middleware basato su funzione** utilizzando il Microsoft Agent Framework. Costruiamo sull'esempio di workflow condizionale aggiungendo uno strato di middleware che concede privilegi speciali ai membri prioritari.

## Cosa Imparerai:
1. **Middleware basato su funzione**: Intercettare e modificare i risultati delle funzioni
2. **Accesso al contesto**: Leggere e modificare `context.result` dopo l'esecuzione
3. **Implementazione della logica di business**: Vantaggi per i membri prioritari
4. **Sovrascrittura del risultato**: Cambiare gli esiti delle funzioni in base allo status dell'utente
5. **Stesso workflow, esiti diversi**: Variazioni comportamentali guidate dal middleware

## Architettura del Workflow con Middleware:

```
User Input: "I want to book a hotel in Paris"
                    ↓
        [availability_agent]
        - Calls hotel_booking tool
        - 🌟 priority_check middleware intercepts
        - Checks user membership status
        - IF priority + no rooms → Override to available!
        - Returns BookingCheckResult
                    ↓
        Conditional Routing
           /                    \
    [has_availability]    [no_availability]
          ↓                      ↓
    [booking_agent]        [alternative_agent]
    (Priority override!)   (Regular users)
          ↓                      ↓
       [display_result executor]
```

## Differenza Chiave rispetto al Workflow Condizionale:

**Senza Middleware** (14-conditional-workflow.ipynb):
- Parigi non ha camere → Instradamento a alternative_agent

**Con Middleware** (questo notebook):
- Utente normale + Parigi → Nessuna camera → Instradamento a alternative_agent
- Utente prioritario + Parigi → 🌟 Middleware sovrascrive! → Disponibile → Instradamento a booking_agent

## Prerequisiti:
- Microsoft Agent Framework installato
- Comprensione dei workflow condizionali (vedi 14-conditional-workflow.ipynb)
- Token GitHub o chiave API OpenAI
- Nozioni di base sui pattern di middleware


In [1]:
import asyncio
import json
import os
from collections.abc import Awaitable, Callable
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    FunctionInvocationContext,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## Passo 1: Definire i Modelli Pydantic per Output Strutturati

Questi modelli definiscono lo **schema** che gli agenti restituiranno. Abbiamo aggiunto un campo `priority_override` per tracciare quando il middleware modifica il risultato della disponibilità.


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str
    priority_override: bool = False  # 🆕 NEW! Tracks if middleware overrode the result


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check with priority_override)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check with priority_override)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## Step 2: Definire il Database dei Membri Prioritari

Per questa demo, simuleremo un database di membri prioritari. In produzione, questo interrogerebbe un vero database o API.

**Membri Prioritari:**
- `alice@example.com` - Membro VIP
- `bob@example.com` - Membro Premium  
- `priority_user` - Account di test


In [3]:
# Simulated priority members database
PRIORITY_MEMBERS = {
    "alice@example.com",
    "bob@example.com",
    "priority_user",
}

# Global variable to track current user (in real app, use proper session management)
current_user_id = "regular_user"  # Default: regular user


def set_user(user_id: str):
    """Set the current user for the session."""
    global current_user_id
    current_user_id = user_id
    is_priority = user_id in PRIORITY_MEMBERS
    status = "🌟 PRIORITY MEMBER" if is_priority else "👤 Regular User"

    display(
        HTML(f"""
        <div style='padding: 15px; background: {"linear-gradient(135deg, #FFD700 0%, #FFA500 100%)" if is_priority else "#e3f2fd"}; 
                    border-left: 4px solid {"#FF6B35" if is_priority else "#2196f3"}; border-radius: 4px; margin: 10px 0;'>
            <strong>👤 Current User Set:</strong> {user_id}<br>
            <strong>Status:</strong> {status}
        </div>
    """)
    )


print("✅ Priority members database created")
print(f"   Priority members: {len(PRIORITY_MEMBERS)} users")

✅ Priority members database created
   Priority members: 3 users


## Step 3: Crea lo Strumento di Prenotazione Hotel

Stesso flusso di lavoro condizionale, ma ora verrà intercettato dal middleware!


In [4]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Step 4: 🌟 Crea il Middleware di Controllo Priorità (LA CARATTERISTICA CHIAVE!)

Questa è la **funzionalità principale** di questo notebook. Il middleware:

1. **Intercetta** la chiamata della funzione hotel_booking
2. **Esegue** normalmente la funzione chiamando `next(context)`
3. **Ispeziona** il risultato in `context.result`
4. **Sovrascrive** il risultato se l'utente è prioritario e non ci sono camere disponibili
5. **Ritorna** il risultato modificato all'agente

**Schema Chiave:**
```python
async def my_middleware(context, next):
    await next(context)  # Esegui funzione
    # Ora context.result contiene l'output della funzione
    if some_condition:
        context.result = new_value  # Sovrascrivere!
```


In [5]:
async def priority_check_middleware(
    context: FunctionInvocationContext,
    next: Callable[[FunctionInvocationContext], Awaitable[None]],
) -> None:
    """
    Function middleware that overrides hotel_booking results for priority members.
    
    Workflow:
    1. Let the function execute normally
    2. Check if user is a priority member
    3. If priority + no availability → Override to make rooms available!
    4. Agent will then route to booking path instead of alternative path
    """
    function_name = context.function.name

    display(
        HTML(f"""
        <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Middleware:</strong> Intercepting {function_name}...
        </div>
    """)
    )

    # Execute the original function
    await next(context)

    # Now inspect and potentially modify the result
    if context.result and function_name == "hotel_booking":
        result_data = json.loads(context.result)
        destination = result_data.get("destination", "")
        has_availability = result_data.get("has_availability", False)

        # Check if user is priority member
        is_priority = current_user_id in PRIORITY_MEMBERS

        # Override logic: Priority member + no availability → Make available!
        if is_priority and not has_availability:
            display(
                HTML(f"""
                <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); 
                            border-radius: 8px; margin: 10px 0; box-shadow: 0 4px 12px rgba(255,165,0,0.4);'>
                    <h3 style='margin: 0 0 10px 0; color: #333;'>🌟 PRIORITY OVERRIDE ACTIVATED! 🌟</h3>
                    <p style='margin: 0; color: #555; line-height: 1.6;'>
                        <strong>User:</strong> {current_user_id}<br>
                        <strong>Status:</strong> VIP Priority Member<br>
                        <strong>Action:</strong> Overriding "No Availability" for {destination}<br>
                        <strong>Result:</strong> ✅ Rooms now available for priority booking!
                    </p>
                </div>
            """)
            )

            # Override the result!
            result_data["has_availability"] = True
            result_data["priority_override"] = True
            context.result = json.dumps(result_data)

        elif not has_availability:
            display(
                HTML(f"""
                <div style='padding: 12px; background: #ffebee; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>ℹ️ Middleware:</strong> No priority override (user: {current_user_id})
                </div>
            """)
            )


print("✅ priority_check_middleware created")
print("   - Intercepts hotel_booking function")
print("   - Overrides availability for priority members")

✅ priority_check_middleware created
   - Intercepts hotel_booking function
   - Overrides availability for priority members


## Step 5: Definire le Funzioni di Condizione per il Routing

Stesse funzioni di condizione del workflow condizionale - ispezionano l'output strutturato per determinare il routing.


In [6]:
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available (including priority overrides!)."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        # Check if this was a priority override
        override_indicator = " 🌟" if result.priority_override else ""

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}{override_indicator}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception:
        return False


print("✅ Condition functions defined")

✅ Condition functions defined


## Passo 6: Creare un Executor di Visualizzazione Personalizzato

Stesso executor di prima - visualizza l'output finale del workflow.


In [7]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """Display the final result as workflow output."""
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created")

✅ display_result executor created


## Passo 7: Caricare le Variabili d'Ambiente

Configura il client LLM (Modelli GitHub o OpenAI).


In [8]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")


## Passo 8: Creare Agenti AI con Middleware

**DIFFERENZA PRINCIPALE:** Quando creiamo l'availability_agent, passiamo il parametro `middleware`!

Ecco come iniettiamo il priority_check_middleware nella pipeline di invocazione della funzione dell'agente.


In [9]:
# Agent 1: Check availability with tool + middleware
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), message (string), "
            "and priority_override (bool, true if priority member got special access). "
            "The message should summarize the availability status and mention if priority override occurred."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
        middleware=[priority_check_middleware],  # 🌟 MIDDLEWARE INJECTION!
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "If priority_override is true in the input, mention that they received priority member access. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - WITH priority_check_middleware 🌟</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## Passo 9: Costruisci il Workflow

Stessa struttura del workflow di prima - instradamento condizionale basato sulla disponibilità.


In [10]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH (can be triggered by middleware override!)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing (Middleware-Aware):</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> (or 🌟 <strong>priority override</strong>) → booking_agent → display_result
        </p>
    </div>
""")
)

## Step 10: Caso di Test 1 - Utente Regolare a Parigi (Nessuna Sovrascrittura)

Un utente regolare tenta di prenotare Parigi → Nessuna stanza → Instradamento verso alternative_agent


In [11]:
# Set as regular user
set_user("regular_user")

display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Regular User + Paris</h3>
        <p style='margin: 0;'><strong>Expected:</strong> No rooms → No middleware override → Alternative suggestion</p>
    </div>
""")
)

# Create request
request_regular = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_regular = await workflow.run(request_regular)
outputs_regular = events_regular.get_outputs()

# Display results
if outputs_regular:
    result_regular = AlternativeResult.model_validate_json(outputs_regular[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: #fff; border: 2px solid #ff9800; border-radius: 12px; margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #e65100;'>📊 RESULT (Regular User)</h3>
            <div style='background: #fff3e0; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0;'><strong>Middleware:</strong> No priority override (regular user)</p>
                <p style='margin: 0 0 10px 0;'><strong>Alternative:</strong> 🏨 {result_regular.alternative_destination}</p>
                <p style='margin: 0;'><strong>Reason:</strong> {result_regular.reason}</p>
            </div>
        </div>
    """)
    )

## Step 11: Test Case 2 - 🌟 Utente Prioritario a Parigi (CON Override!)

Un membro prioritario prova a prenotare Parigi → Nessuna stanza inizialmente → 🌟 Il middleware sovrascrive! → Inoltra a booking_agent

**Questa è la dimostrazione chiave del potere del middleware!**


In [12]:
# Set as priority user
set_user("priority_user")

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #333;'>🧪 TEST CASE 2: 🌟 Priority User + Paris</h3>
        <p style='margin: 0; color: #555;'><strong>Expected:</strong> No rooms → 🌟 MIDDLEWARE OVERRIDE → Rooms available → Booking suggestion!</p>
    </div>
""")
)

# Create request
request_priority = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run workflow
events_priority = await workflow.run(request_priority)
outputs_priority = events_priority.get_outputs()

# Display results
if outputs_priority:
    result_priority = BookingConfirmation.model_validate_json(outputs_priority[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; 
                    box-shadow: 0 8px 16px rgba(255,165,0,0.4); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 RESULT (Priority Member) 🌟</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Priority Override!)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> 🌟 OVERRIDE ACTIVATED!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_priority.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_priority.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_priority.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #fff3cd; border-radius: 6px; border-left: 4px solid #FF6B35;'>
                    <strong>💡 What Just Happened:</strong><br>
                    1. hotel_booking tool returned "no availability"<br>
                    2. priority_check_middleware intercepted the result<br>
                    3. Middleware checked user status: priority_user ✅<br>
                    4. Middleware OVERRODE the result to "available"<br>
                    5. Workflow routed to booking_agent instead of alternative_agent!
                </div>
            </div>
        </div>
    """)
    )

## Step 12: Caso di test 3 - Utente prioritario a Stoccolma (Già disponibile)

L'utente prioritario prova Stoccolma → Stanze disponibili → Nessuna sostituzione necessaria → Instradamento verso booking_agent

Questo mostra che il middleware agisce solo quando necessario!


In [13]:
# Priority user is still set from previous test

display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 3: Priority User + Stockholm</h3>
        <p style='margin: 0;'><strong>Expected:</strong> Rooms available → No override needed → Booking suggestion</p>
    </div>
""")
)

# Create request
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; 
                    box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 RESULT (Priority User - No Override Needed)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available (Natural)</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Middleware:</strong> No override needed</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <div style='margin-top: 15px; padding: 15px; background: #e8f5e9; border-radius: 6px; border-left: 4px solid #4caf50;'>
                    <strong>💡 Middleware Behavior:</strong><br>
                    • hotel_booking returned "available" naturally<br>
                    • Middleware saw available = true → No override needed<br>
                    • Workflow proceeded normally to booking_agent
                </div>
            </div>
        </div>
    """)
    )

## Punti Chiave e Concetti di Middleware

### ✅ Cosa Hai Imparato:

#### **1. Pattern Middleware Basato su Funzioni**

Il middleware intercetta le chiamate di funzione utilizzando una semplice funzione async:

```python
async def my_middleware(
    context: FunctionInvocationContext,
    next: Callable,
) -> None:
    # Prima dell'esecuzione della funzione
    print("Intercepting...")
    
    # Esegui la funzione
    await next(context)
    
    # Dopo l'esecuzione della funzione - ispeziona il risultato
    if context.result:
        # Modifica il risultato se necessario
        context.result = modified_value
```

#### **2. Accesso al Contesto e Sovrascrittura del Risultato**

- `context.function` - Accede alla funzione chiamata
- `context.arguments` - Legge gli argomenti della funzione
- `context.kwargs` - Accede ai parametri aggiuntivi
- `await next(context)` - Esegue la funzione
- `context.result` - Legge/modifica l'output della funzione

#### **3. Implementazione della Logica di Business**

Il nostro middleware implementa i benefici per i membri priority:
- **Utenti regolari**: Nessuna modifica, flusso standard
- **Utenti priority**: Sovrascrive "nessuna disponibilità" → "disponibile"
- **Logica condizionale**: Sovrascrive solo quando necessario

#### **4. Stesso Flusso di Lavoro, Risultati Diversi**

Il potere del middleware:
- ✅ Nessuna modifica alla struttura del flusso di lavoro
- ✅ Nessuna modifica alla funzione dello strumento
- ✅ Nessuna modifica alla logica di routing condizionale
- ✅ Solo middleware → Comportamento diverso!

### 🚀 Applicazioni Nel Mondo Reale:

1. **Funzionalità VIP/Premium**
   - Sovrascrivere i limiti di rate per utenti premium
   - Fornire accesso prioritario alle risorse
   - Sbloccare funzionalità premium dinamicamente

2. **A/B Testing**
   - Indirizzare utenti a implementazioni diverse
   - Testare nuove funzionalità con utenti specifici
   - Rilasci graduali di funzionalità

3. **Sicurezza e Conformità**
   - Audit delle chiamate di funzione
   - Bloccare operazioni sensibili
   - Applicare regole di business

4. **Ottimizzazione delle Prestazioni**
   - Cache dei risultati per utenti specifici
   - Saltare operazioni costose quando possibile
   - Allocazione dinamica delle risorse

5. **Gestione Errori e Retry**
   - Catturare e gestire errori con garbo
   - Implementare logica di retry
   - Ripiegare su implementazioni alternative

6. **Logging e Monitoraggio**
   - Tracciare tempi di esecuzione delle funzioni
   - Registrare parametri e risultati
   - Monitorare i pattern di utilizzo

### 🔑 Differenze Chiave Rispetto ai Decoratori:

| Caratteristica | Decoratore | Middleware |
|----------------|------------|------------|
| **Ambito** | Singola funzione | Tutte le funzioni nell'agente |
| **Flessibilità** | Fissa alla definizione | Dinamica a runtime |
| **Contesto** | Limitato | Contesto completo dell'agente |
| **Composizione** | Molteplici decoratori | Pipeline di middleware |
| **Consapevolezza dell'Agente** | No | Sì (accesso allo stato agente) |

### 📚 Quando Usare il Middleware:

✅ **Usa il middleware quando:**
- Devi modificare il comportamento in base allo stato utente/sessione
- Vuoi applicare logica a più funzioni
- Hai bisogno di accesso al contesto a livello di agente
- Stai implementando aspetti trasversali (logging, autenticazione, ecc.)

❌ **Non usare il middleware quando:**
- Validazione semplice dell'input (usa Pydantic)
- Logica specifica di una funzione (tienila nella funzione)
- Modifiche una tantum (modifica direttamente la funzione)

### 🎓 Pattern Avanzati:

```python
# Più middleware (l'ordine di esecuzione è importante!)
middleware=[
    logging_middleware,      # Prima i log
    auth_middleware,         # Poi controlla l'autenticazione
    cache_middleware,        # Poi controlla la cache
    rate_limit_middleware,   # Poi limita la velocità
    priority_check_middleware  # Infine controllo priorità
]

# Esecuzione condizionale del middleware
async def conditional_middleware(context, next):
    if should_execute(context):
        await next(context)
        # Modifica il risultato
    else:
        # Salta completamente l'esecuzione
        context.result = cached_value
```

### 🔗 Concetti Correlati:

- **Agent Middleware**: Intercetta le chiamate agent.run()
- **Function Middleware**: Intercetta le chiamate di funzioni dello strumento (quello che abbiamo usato!)
- **Middleware Pipeline**: Catena di middleware che vengono eseguiti in ordine
- **Propagation del Contesto**: Passaggio dello stato attraverso la catena di middleware


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Avvertenza**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Pur impegnandoci per garantire accuratezza, si prega di considerare che le traduzioni automatiche possono contenere errori o inesattezze. Il documento originale nella sua lingua nativa deve essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un essere umano. Non ci assumiamo alcuna responsabilità per eventuali malintesi o interpretazioni errate derivanti dall’uso di questa traduzione.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
